In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from time import perf_counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
# Create a list with the filepaths for training and testing
ext = ['jpeg', 'jpg', 'png']    # Add image formats here

files = []
file_paths = []
dir_ = Path("C:\\Users\\rizky\\Bangkit2021\\Project\\SIBI_SL\\test_new\\GestureTest")
[files.extend(list(dir_.glob(r"**/*" + e )) for e in ext)]
for i in range(len(files)):
    file_paths.extend(files[i])
print(len(file_paths))

39000


In [7]:
#Number of pictures and different labels in dataset
def proc_img(filepath):

    labels = [str(filepath[i]).split("\\")[-2] \
              for i in range(len(filepath))]
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    print(labels.unique())
    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return df

df = proc_img(file_paths)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [8]:
train_df, test_df = train_test_split(df,test_size=0.1, random_state=42)

In [37]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    validation_split=0.1
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    subset='training',
    brightness_range=[0.7,1.3]
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    subset='validation',
    brightness_range=[0.7,1.3]
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 31590 validated image filenames belonging to 26 classes.
Found 3510 validated image filenames belonging to 26 classes.
Found 3609 validated image filenames belonging to 24 classes.


In [24]:
model = tf.keras.models.load_model("C:\\Users\\rizky\\Bangkit2021\\Project\\Capstone-SIBI\\ml\\ModelMnistCNN\\my_model.h5")

In [39]:
pred = model.predict(test_images)

InvalidArgumentError:  Input to reshape is a tensor with 9331200 values, but the requested shape requires a multiple of 2048
	 [[node sequential/flatten/Reshape (defined at <ipython-input-25-107ed4412b41>:1) ]] [Op:__inference_predict_function_19292]

Function call stack:
predict_function


In [28]:
pred = np.argmax(pred,axis=1)

In [29]:
# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Get the accuracy on the test set
y_test = list(test_df.Label)
acc = accuracy_score(y_test,pred)

# Display a confusion matrix
cf_matrix = confusion_matrix(y_test, pred, normalize='true')
plt.figure(figsize = (17,12))
sns.heatmap(cf_matrix, annot=True, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)),cbar=False)
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.xticks(fontsize=12,rotation=45)
plt.yticks(fontsize=12)
plt.savefig("conf_matrix2.png")
plt.show()


ValueError: Found input variables with inconsistent numbers of samples: [3900, 710775]

In [22]:
score = model.evaluate(test_images)

122/122 [==============================] - 21s 160ms/step - loss: 12.8395 - accuracy: 0.3185
